In [5]:
from custom_gridworld.gridworld_v0 import env

# Créez une instance de l'environnement avec les paramètres souhaités
environment = env(grid_size=5, num_agents=2, max_steps=200)

# Réinitialisez l'environnement
obs, infos = environment.reset()
print("Observations initiales:", obs)

# Affichez l'état initial
environment.render()

# Effectuez une action aléatoire par agent
actions = {agent: environment.action_space(agent).sample() for agent in environment.agents}
print("actions :", actions) 
obs, rewards, terminations, truncations, infos = environment.step(actions)

print("Observations après une action:", obs)
print("Récompenses:", rewards)
print("Terminaisons:", terminations)
print("Troncations:", truncations)

environment.render()

Observations initiales: {'agent_0': array([0, 0, 0, 0, 4, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0]), 'agent_1': array([0, 0, 0, 0, 4, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])}
[['A' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '_' 'G']]
actions : {'agent_0': np.int64(1), 'agent_1': np.int64(0)}
Observations après une action: {'agent_0': array([1, 0, 0, 0, 4, 4, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0]), 'agent_1': array([1, 0, 0, 0, 4, 4, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])}
Récompenses: {'agent_0': 0, 'agent_1': 0}
Terminaisons: {'agent_0': False, 'agent_1': False}
Troncations: {'agent_0': False, 'agent_1': False}
[['A' 'A' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '_' 'G']]
